In [1]:
import os 
cwd = os.getcwd()

os.chdir('../')
cwd = os.getcwd()
print(cwd)
 
os.chdir('./code/')
cwd = os.getcwd()
print(cwd)

import pandas as pd 
import numpy as np 
import seaborn as sns

import pickle 
import logging


from config.config import SQLQuery
querySno = SQLQuery('snowflake')

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [2]:
import sys
sys.path.insert(0, '/Users/sarfrazahmed/Documents/projects/deposit-model/conf')
import pandas as pd
import numpy as np
import chart_studio.plotly as py 
from datetime import timedelta
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
import plotly.io as pio
from pandas_profiling import ProfileReport
pio.templates.default = 'plotly_dark'
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
color_map = {'high': '#ef553b', 'low': '#636efa'}
import datetime
import pytz

In [5]:
querySno = SQLQuery('snowflake')
rev_df = querySno(r"""
with lending_data as (
select business_id, due_date 
FROM (select distinct business_id, due_date from PROD_DB.ADHOC.lending_dpd_data_till_may20) pay

-- and pay.lending_business_id = 'cf6d8b20-574f-4362-b1f5-274bb2231c05'
order by due_date
)

,txns_with_categories AS (
select a.business_id, d.due_date,DATEDIFF(month, TRANSACTION_DATE, d.due_date) AS txn_month, amount, description,
			REGEXP_REPLACE(REGEXP_REPLACE(lower(description), ':',' '), '[^a-z ]', '') AS description_processed, transaction_date,
			            CASE WHEN description_processed ILIKE any(
                                                '%%novo funding%%',
			            						'%% earninactivehours %%', 
                                                'earninactivehours %%',
                                                '%% earninactivehours',
                                                'earninactivehours',
			            						'%% activehours %%', 
                                                'activehours %%', 
                                                '%% activehours', 
                                                'activehours', 
			            						'%% navchek %%', 
                                                'navchek %%', 
                                                '%% navchek', 
                                                'navchek', 
			            						'%% loan %%', 
                                                'loan %%',
                                                '%% loan',
                                                'loan',
			            						'%% earnin %%',
                                                'earnin %%',
                                                '%% earnin',
                                                'earnin', 
			            						'%% advance %%',
                                                '%% advance',
                                                'advance %%',
                                                'advance'
			            ) THEN 1 ELSE 0 END AS LOANS,
			            CASE WHEN description_processed ILIKE any(
			            						'%% hcclaimpmt %%',
                                                '%% hcclaimpmt',
                                                'hcclaimpmt %%',
                                                'hcclaimpmt',
												 '%% unitedhealthcare %%',
                                                 'unitedhealthcare %%',
                                                 '%% unitedhealthcare',
                                                 'unitedhealthcare',
												 '%% bcbs %%',
                                                 '%% bcbs',
                                                 'bcbs %%',
                                                 'bcbs',
												 '%% cigna %%',
                                                 'cigna %%',
                                                 '%% cigna',
                                                 'cigna',
												 '%% govt %%',
                                                 '%% govt',
                                                 'govt %%',
                                                 'govt',
												 '%% humana %%',
                                                 '%% humana',
                                                 'humana %%',
                                                 'humana',
												 '%% aetna %%',
                                                 'aetna %%',
                                                 '%% aetna',
                                                 'aetna',
												 '%% insurance %%',
                                                 'insurance %%',
                                                 '%% insurance',
                                                 'insurance',
												 '%% life %%',
                                                 'life %%',
                                                 '%% life',
                                                 'life',
												 '%% blue %%',
                                                 '%% blue',
                                                 'blue %%',
                                                 'blue'
						) THEN 1 ELSE 0 END AS INSURANCE,
						CASE WHEN description_processed ILIKE any(
												'%% sbtpg %%',
                                                '%% sbtpg',
                                                'sbtpg %%',
                                                'sbtpg', 
												'%% xxtaxeip %%',
                                                'xxtaxeip %%',
                                                '%% xxtaxeip',
                                                'xxtaxeip', 
												'%% tax %%', 
                                                'tax %%', 
                                                '%% tax', 
                                                'tax', 
												'%% irs %%', 
                                                '%% irs', 
                                                'irs %%', 
                                                'irs', 
												'%% treas %%', 
                                                '%% treas', 
                                                'treas %%', 
                                                'treas', 
												'%% taxrfd %%',
                                                '%% taxrfd',
                                                'taxrfd %%',
                                                'taxrfd'
						) THEN 1 ELSE 0 END AS TAX
from fivetran_db.PROD_NOVO_API_PUBLIC.transactions a inner join lending_data d on a.business_id = d.business_id
		   WHERE datediff(MONTH ,TRANSACTION_DATE,d.due_date) < 2 and datediff(MONTH ,TRANSACTION_DATE,d.due_date) >= 0
           AND status='active' AND TYPE = 'credit'     
)
,all_credits AS (
SELECT a.BUSINESS_ID, due_date,txn_month,
               sum(amount)  AS total_credit_amount, 
               sum(CASE WHEN LOANS = 1 THEN amount ELSE 0 end) AS total_loan_credit_amount, 
               sum(CASE WHEN INSURANCE = 1 THEN amount ELSE 0 END) AS total_insurance_credit_amount, 
               sum(CASE WHEN TAX = 1 THEN amount ELSE 0 end) AS total_tax_credit_amount
FROM  txns_with_categories a
GROUP BY 1,2,3
),
all_txn_credits AS (
SELECT business_id, due_date,txn_month, 
               COALESCE(total_credit_amount, 0) AS total_credit_amount,
               COALESCE(total_loan_credit_amount, 0) AS total_loan_credit_amount,
               COALESCE(total_insurance_credit_amount, 0) AS total_insurance_credit_amount,
               COALESCE(total_tax_credit_amount, 0) AS total_tax_credit_amount
            --   COALESCE(TOTAL_CREDIT_AMOUNT_PLAID, 0) AS TOTAL_CREDIT_AMOUNT_PLAID,
            --   COALESCE(TOTAL_LOAN_CREDIT_AMT_PLAID, 0) AS TOTAL_LOAN_CREDIT_AMT_PLAID,
            --   COALESCE(TOTAL_INSURANCE_CREDIT_AMT_PLAID, 0) AS TOTAL_INSURANCE_CREDIT_AMT_PLAID,
            --   COALESCE(TOTAL_TAX_CREDIT_AMT_PLAID, 0) AS TOTAL_TAX_CREDIT_AMT_PLAID
FROM all_credits 
)
,last_6mnth_revenue as (
SELECT business_id, due_date,
       CASE WHEN txn_month = 0 THEN 'past_1month_rev'
            WHEN txn_month = 1 THEN 'past_2month_rev'
            WHEN txn_month = 2 THEN 'past_3month_rev'
            WHEN txn_month in (4,5,6) THEN 'past_furthest_3months'
       END as month,
       ((total_credit_amount-total_loan_credit_amount-total_insurance_credit_amount-total_tax_credit_amount)
	           ) AS revenue
FROM all_txn_credits
where txn_month <= 1)

SELECT business_id, due_date, revenue_past_1m, revenue_past_2m, revenue_past_1m + revenue_past_2m as total_revenue_2m, revenue_past_1m/(1 + revenue_past_2m) as ratio_revenue_1m_2m from (
select business_id,due_date, revenue as revenue_past_1m, lead(revenue) over(order by business_id, due_date, month) as revenue_past_2m,
row_number() over(partition by business_id, due_date order by month) rnk
from (
select business_id, due_date, month, sum(revenue) revenue
from last_6mnth_revenue
-- where business_id = '0d39a85e-12dc-4487-904a-55cc24c397bf'
group by 1,2,3
order by 1,2,3
)
)
where rnk = 1
"""
)

In [6]:
rev_df.head()

,business_id,due_date,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m
0,b1f8dd2b-a953-4814-9b2b-e44675c17b1c,2023-05-10,22550.62,12359.29,34909.91,1.824441
1,6123b27a-67b1-4a5f-8b5c-c4fb3a1104b7,2023-04-13,8778.02,11758.95,20536.97,0.746433
2,84d80cf9-adc2-4e04-9248-0c6dc5a398b7,2022-11-02,654.33,529.41,1183.74,1.233631
3,ac232438-12d9-4eda-9d47-e189fa9729c3,2023-05-10,3477.56,4674.62,8152.18,0.743764
4,6c45c434-a32a-499e-9155-3d48c1724649,2023-05-13,12279.82,12799.46,25079.28,0.959326


In [7]:
rev_df.shape

(7913, 6)

In [8]:
txn_df = querySno(r"""
select business_id, due_date, 
avg(count_pd_deposit_1m)count_pd_deposit_1m, avg(count_ach_debit_1m)count_ach_debit_1m, avg(count_ach_credit_1m)count_ach_credit_1m, avg(count_card_withdrawal_1m)count_card_withdrawal_1m, avg(count_mrdc_credit_1m)count_mrdc_credit_1m,
avg(count_pd_deposit_2m)count_pd_deposit_2m, avg(count_ach_debit_2m)count_ach_debit_2m, avg(count_ach_credit_2m)count_ach_credit_2m, avg(count_card_withdrawal_2m)count_card_withdrawal_2m, avg(count_mrdc_credit_2m)count_mrdc_credit_2m,
avg(amount_pd_deposit_1m)amount_pd_deposit_1m, avg(amount_ach_debit_1m)amount_ach_debit_1m, avg(amount_ach_credit_1m)amount_ach_credit_1m, avg(amount_card_withdrawal_1m)amount_card_withdrawal_1m, avg(amount_mrdc_credit_1m)amount_mrdc_credit_1m, 
avg(amount_pd_deposit_2m)amount_pd_deposit_2m, avg(amount_ach_debit_2m)amount_ach_debit_2m, avg(amount_ach_credit_2m)amount_ach_credit_2m, avg(amount_card_withdrawal_2m)amount_card_withdrawal_2m, avg(amount_mrdc_credit_2m)amount_mrdc_credit_2m,
avg(pd_deposit_size_1m)pd_deposit_size_1m, avg(ach_debit_size_1m)ach_debit_size_1m, avg(ach_credit_size_1m)ach_credit_size_1m, avg(card_withdrawal_size_1m)card_withdrawal_size_1m, avg(mrdc_credit_size_1m)mrdc_credit_size_1m, 
avg(pd_deposit_size_2m)pd_deposit_size_2m, avg(ach_debit_size_2m)ach_debit_size_2m, avg(ach_credit_size_2m)ach_credit_size_2m, avg(card_withdrawal_size_2m)card_withdrawal_size_2m, avg(mrdc_credit_size_2m)mrdc_credit_size_2m,
avg(amount_credited_1m)amount_credited_1m, avg(distinct_credit_txns_1m)distinct_credit_txns_1m, avg(credit_size_1m)credit_size_1m, avg(median_amount_credited_1m)median_amount_credited_1m, avg(stddev_amount_credited_1m)stddev_amount_credited_1m, 
avg(amount_credited_2m)amount_credited_2m, avg(distinct_credit_txns_2m)distinct_credit_txns_2m, avg(credit_size_2m)credit_size_2m, avg(median_amount_credited_2m)median_amount_credited_2m, avg(stddev_amount_credited_2m)stddev_amount_credited_2m,
avg(amount_debited_1m)amount_debited_1m, avg(distinct_debit_txns_1m)distinct_debit_txns_1m, avg(debit_size_1m)debit_size_1m, avg(median_amount_debited_1m)median_amount_debited_1m, avg(stddev_amount_debited_1m)stddev_amount_debited_1m, 
avg(amount_debited_2m)amount_debited_2m, avg(distinct_debit_txns_2m)distinct_debit_txns_2m, avg(debit_size_2m)debit_size_2m, avg(median_amount_debited_2m)median_amount_debited_2m, avg(stddev_amount_debited_2m)stddev_amount_debited_2m,
avg(ratio_debit_credit_1m)ratio_debit_credit_1m, avg(ratio_debit_credit_frequency_1m)ratio_debit_credit_frequency_1m, avg(ratio_debit_credit_2m)ratio_debit_credit_2m, avg(ratio_debit_credit_frequency_2m)ratio_debit_credit_frequency_2m,
avg(ratio_pd_amt_total_credit_1m)ratio_pd_amt_total_credit_1m, avg(ratio_ach_debit_total_debit_1m)ratio_ach_debit_total_debit_1m, avg(ratio_ach_credit_total_credit_1m)ratio_ach_credit_total_credit_1m, avg(ratio_card_withdrawal_total_debit_1m)ratio_card_withdrawal_total_debit_1m, avg(ratio_mrdc_credit_total_credit_1m)ratio_mrdc_credit_total_credit_1m, 
avg(ratio_pd_amt_total_credit_2m)ratio_pd_amt_total_credit_2m, avg(ratio_ach_debit_total_debit_2m)ratio_ach_debit_total_debit_2m, avg(ratio_ach_credit_total_credit_2m)ratio_ach_credit_total_credit_2m, avg(ratio_card_withdrawal_total_debit_2m)ratio_card_withdrawal_total_debit_2m, avg(ratio_mrdc_credit_total_credit_2m)ratio_mrdc_credit_total_credit_2m,
avg(ratio_pd_freq_total_credit_1m)ratio_pd_freq_total_credit_1m, avg(ratio_ach_debit_freq_total_debit_1m)ratio_ach_debit_freq_total_debit_1m, avg(ratio_ach_credit_freq_total_credit_1m)ratio_ach_credit_freq_total_credit_1m, avg(ratio_card_withdrawal_freq_total_debit_1m)ratio_card_withdrawal_freq_total_debit_1m, avg(ratio_mrdc_credit_freq_total_credit_1m)ratio_mrdc_credit_freq_total_credit_1m, 
avg(ratio_pd_freq_total_credit_2m)ratio_pd_freq_total_credit_2m, avg(ratio_ach_debit_freq_total_debit_2m)ratio_ach_debit_freq_total_debit_2m, avg(ratio_ach_credit_freq_total_credit_2m)ratio_ach_credit_freq_total_credit_2m, avg(ratio_card_withdrawal_freq_total_debit_2m)ratio_card_withdrawal_freq_total_debit_2m, avg(ratio_mrdc_credit_freq_total_credit_2m)ratio_mrdc_credit_freq_total_credit_2m,
avg(ratio_ach_debit_credit_1m)ratio_ach_debit_credit_1m, avg(ratio_ach_debit_credit_2m)ratio_ach_debit_credit_2m,
avg(ratio_ach_debit_credit_freq_1m)ratio_ach_debit_credit_freq_1m, avg(ratio_ach_debit_credit_freq_2m)ratio_ach_debit_credit_freq_2m,
avg(ach_d_size_1m)ach_d_size_1m, avg(ach_d_size_2m)ach_d_size_2m, avg(ach_c_size_1m)ach_c_size_1m, avg(ach_c_size_2m)ach_c_size_2m


from (
select pay.business_id, pay.due_date, t.transaction_date, t.amount, t.type, t.running_balance, pay.min_created_at,
datediff(day, t.transaction_date, pay.due_date) as DaysFromDueDate, datediff(day, pay.min_created_at, t.transaction_date) as DaysSinceLoanTaken,
CASE WHEN TYPE = 'credit' and DESCRIPTION  ILIKE ANY(
	                    '%%AMAZON%%'
	                    , '%%AMZN%%'
	                    , '%%STRIPE%%'
	                    , '%%SQUARE INC%%'
	                    , '%%SHOPIFY%%'
	                    , '%%SHOPPAY%%'
	                    )
                    AND NOT(
	                    description ILIKE ANY(
	                        '%%LYFT'
	                        , '%%OfferUp%%'
	                        , '%%Gumroad%%'
	                        , '%%FB%%Fundrai%%'
	                        , '%%Verify%%'
	                        , '%%CASH%%'
	                        , '%%PAYROLL%%'
	                        , '%%VRFY%%'
	                        , '%%CAPITAL%%'
	                        , '%%REFUND%%'
	                    ) ) THEN 1 ELSE 0 END as PD_DEPOSIT_TXN,
CASE WHEN TYPE ='debit' AND MEDIUM  ILIKE ANY('%%external withdrawal%%', '%%iat withdrawal%%') THEN 1 ELSE 0 END AS ach_debit_txn,
CASE WHEN type = 'credit'  AND medium ILIKE ANY('%%external deposit%%', '%%iat deposit%%') THEN 1 ELSE 0 END AS ach_credit_txn,
CASE WHEN medium = 'POS Withdrawal' THEN 1 ELSE 0 END AS card_withdrawal_txn,
CASE WHEN TYPE = 'credit' AND description='Mobile Check Deposit' THEN 1 ELSE 0 END AS mrdc_credit_txn,

sum(case when daysFromDueDate <= 30 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_1m,
sum(case when daysFromDueDate <= 30 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_1m,
sum(case when daysFromDueDate <= 30 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_1m,
sum(case when daysFromDueDate <= 30 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_1m,
sum(case when daysFromDueDate <= 30 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_1m,

sum(case when daysFromDueDate <= 60 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_2m,
sum(case when daysFromDueDate <= 60 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_2m,
sum(case when daysFromDueDate <= 60 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_2m,
sum(case when daysFromDueDate <= 60 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_2m,
sum(case when daysFromDueDate <= 60 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_2m,

sum(case when daysFromDueDate <= 90 then PD_DEPOSIT_TXN else 0 end) over (partition by biz.id, due_date) as count_pd_deposit_3m,
sum(case when daysFromDueDate <= 90 then ach_debit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_debit_3m,
sum(case when daysFromDueDate <= 90 then ach_credit_txn else 0 end) over (partition by biz.id, due_date) as count_ach_credit_3m,
sum(case when daysFromDueDate <= 90 then card_withdrawal_txn else 0 end) over (partition by biz.id, due_date) as count_card_withdrawal_3m,
sum(case when daysFromDueDate <= 90 then mrdc_credit_txn else 0 end) over (partition by biz.id, due_date) as count_mrdc_credit_3m,

sum(case when daysFromDueDate <= 30 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_1m,
sum(case when daysFromDueDate <= 30 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_1m,
sum(case when daysFromDueDate <= 30 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_1m,
sum(case when daysFromDueDate <= 30 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_1m,
sum(case when daysFromDueDate <= 30 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_1m,

sum(case when daysFromDueDate <= 60 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_2m,
sum(case when daysFromDueDate <= 60 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_2m,
sum(case when daysFromDueDate <= 60 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_2m,
sum(case when daysFromDueDate <= 60 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_2m,
sum(case when daysFromDueDate <= 60 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_2m,

sum(case when daysFromDueDate <= 90 and PD_DEPOSIT_TXN = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_pd_deposit_3m,
sum(case when daysFromDueDate <= 90 and ach_debit_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_ach_debit_3m,
sum(case when daysFromDueDate <= 90 and ach_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_ach_credit_3m,
sum(case when daysFromDueDate <= 90 and card_withdrawal_txn = 1 then -amount else 0 end) over (partition by biz.id, due_date) as amount_card_withdrawal_3m,
sum(case when daysFromDueDate <= 90 and mrdc_credit_txn = 1 then amount else 0 end) over (partition by biz.id, due_date) as amount_mrdc_credit_3m,

amount_pd_deposit_1m/(1 + count_pd_deposit_1m) as pd_deposit_size_1m, amount_pd_deposit_2m/(1 + count_pd_deposit_2m) as pd_deposit_size_2m, amount_pd_deposit_3m/(1 + count_pd_deposit_3m) as pd_deposit_size_3m,
amount_ach_debit_1m/(count_ach_debit_1m + 1) as ach_debit_size_1m, amount_ach_debit_2m/(count_ach_debit_2m + 1) as ach_debit_size_2m, amount_ach_debit_3m/(count_ach_debit_3m + 1) as ach_debit_size_3m,
amount_ach_credit_1m/(count_ach_credit_1m + 1) as ach_credit_size_1m, amount_ach_credit_2m/(count_ach_credit_2m + 1) as ach_credit_size_2m, amount_ach_credit_3m/(count_ach_credit_3m + 1) as ach_credit_size_3m,
amount_card_withdrawal_1m/(count_card_withdrawal_1m + 1) as card_withdrawal_size_1m, amount_card_withdrawal_2m/(count_card_withdrawal_2m + 1) as card_withdrawal_size_2m, amount_card_withdrawal_3m/(count_card_withdrawal_3m + 1) as card_withdrawal_size_3m,
amount_mrdc_credit_1m/(count_mrdc_credit_1m + 1) as mrdc_credit_size_1m, amount_mrdc_credit_2m/(count_mrdc_credit_2m + 1) as mrdc_credit_size_2m, amount_mrdc_credit_3m/(count_mrdc_credit_3m + 1) as mrdc_credit_size_3m,

sum(case when running_balance < 0 and daysFromDueDate <= 30 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_1m,
sum(case when running_balance = 0 and daysFromDueDate <= 30 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_1m,
avg(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_1m,
median(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_1m,
stddev(case when daysFromDueDate <= 30 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_1m,

sum(case when running_balance < 0 and daysFromDueDate <= 60 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_2m,
sum(case when running_balance = 0 and daysFromDueDate <= 60 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_2m,
avg(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_2m,
median(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_2m,
stddev(case when daysFromDueDate <= 60 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_2m,

sum(case when running_balance < 0 and daysFromDueDate <= 90 then 1 else 0 end) over (partition by biz.id, due_date) as od_count_3m,
sum(case when running_balance = 0 and daysFromDueDate <= 90 then 1 else 0 end) over (partition by biz.id, due_date) as zero_balance_count_3m,
avg(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as avg_running_balance_3m,
median(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as median_running_balance_3m,
stddev(case when daysFromDueDate <= 90 then running_balance else 0 end) over (partition by biz.id, due_date) as stddev_running_balance_3m,

sum(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_1m,
sum(case when daysFromDueDate <= 30 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_1m,
amount_credited_1m/(distinct_credit_txns_1m + 1) as credit_size_1m,
median(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_1m,
stddev(case when daysFromDueDate <= 30 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_1m,

sum(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_1m,
sum(case when daysFromDueDate <= 30 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_1m,
amount_debited_1m/(distinct_debit_txns_1m + 1) as debit_size_1m,
median(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_1m,
stddev(case when daysFromDueDate <= 30 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_1m,

amount_debited_1m/(amount_credited_1m + 1) as ratio_debit_credit_1m,
distinct_debit_txns_1m/(distinct_credit_txns_1m + 1) as ratio_debit_credit_frequency_1m,

sum(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_2m,
sum(case when daysFromDueDate <= 60 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_2m,
amount_credited_2m/(distinct_credit_txns_2m + 1) as credit_size_2m,
median(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_2m,
stddev(case when daysFromDueDate <= 60 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_2m,

sum(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_2m,
sum(case when daysFromDueDate <= 60 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_2m,
amount_debited_2m/(distinct_debit_txns_2m + 1) as debit_size_2m,
median(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_2m,
stddev(case when daysFromDueDate <= 60 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_2m,

amount_debited_2m/(amount_credited_2m + 1) as ratio_debit_credit_2m,
distinct_debit_txns_2m/(distinct_credit_txns_2m + 1) as ratio_debit_credit_frequency_2m,

sum(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as amount_credited_3m,
sum(case when daysFromDueDate <= 90 and type = 'credit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_credit_txns_3m,
amount_credited_3m/(distinct_credit_txns_3m + 1) as credit_size_3m,
median(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as median_amount_credited_3m,
stddev(case when daysFromDueDate <= 90 and type = 'credit' then amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_credited_3m,

sum(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as amount_debited_3m,
sum(case when daysFromDueDate <= 90 and type = 'debit' then 1 else 0 end) over (partition by biz.id, due_date) as distinct_debit_txns_3m,
amount_debited_3m/(distinct_debit_txns_3m + 1) as debit_size_3m,
median(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as median_amount_debited_3m,
stddev(case when daysFromDueDate <= 90 and type = 'debit' then -amount else 0 end) over (partition by biz.id, due_date) as stddev_amount_debited_3m,

amount_debited_3m/(amount_credited_3m + 1) as ratio_debit_credit_3m,
distinct_debit_txns_3m/(distinct_credit_txns_3m + 1) as ratio_debit_credit_frequency_3m,

COALESCE(amount_pd_deposit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_pd_amt_total_credit_1m, COALESCE(amount_pd_deposit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_pd_amt_total_credit_2m,
COALESCE(amount_ach_debit_1m/NULLIF(amount_debited_1m, 0), 0) as ratio_ach_debit_total_debit_1m, COALESCE(amount_ach_debit_2m/NULLIF(amount_debited_2m, 0), 0) as ratio_ach_debit_total_debit_2m,
COALESCE(amount_ach_credit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_ach_credit_total_credit_1m, COALESCE(amount_ach_credit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_ach_credit_total_credit_2m,
COALESCE(amount_card_withdrawal_1m/NULLIF(amount_debited_1m, 0), 0) as ratio_card_withdrawal_total_debit_1m, COALESCE(amount_card_withdrawal_2m/NULLIF(amount_debited_2m, 0), 0) as ratio_card_withdrawal_total_debit_2m,
COALESCE(amount_mrdc_credit_1m/NULLIF(amount_credited_1m, 0), 0) as ratio_mrdc_credit_total_credit_1m, COALESCE(amount_mrdc_credit_2m/NULLIF(amount_credited_2m, 0), 0) as ratio_mrdc_credit_total_credit_2m,

count_pd_deposit_1m/(distinct_credit_txns_1m + 1) as ratio_pd_freq_total_credit_1m, count_pd_deposit_2m/(distinct_credit_txns_2m + 1) as ratio_pd_freq_total_credit_2m,
count_ach_debit_1m/(distinct_debit_txns_1m + 1) as ratio_ach_debit_freq_total_debit_1m, count_ach_debit_2m/(distinct_debit_txns_2m + 1) as ratio_ach_debit_freq_total_debit_2m,
count_ach_credit_1m/(distinct_credit_txns_1m + 1) as ratio_ach_credit_freq_total_credit_1m, count_ach_credit_2m/(distinct_credit_txns_2m + 1) as ratio_ach_credit_freq_total_credit_2m,
count_card_withdrawal_1m/(distinct_debit_txns_1m + 1) as ratio_card_withdrawal_freq_total_debit_1m, count_card_withdrawal_2m/(distinct_debit_txns_2m + 1) as ratio_card_withdrawal_freq_total_debit_2m,
count_mrdc_credit_1m/(distinct_credit_txns_1m + 1) as ratio_mrdc_credit_freq_total_credit_1m, count_mrdc_credit_2m/(distinct_credit_txns_2m + 1) as ratio_mrdc_credit_freq_total_credit_2m,

amount_ach_debit_1m/(amount_ach_credit_1m + 1) as ratio_ach_debit_credit_1m, amount_ach_debit_2m/(amount_ach_credit_2m + 1) as ratio_ach_debit_credit_2m,
count_ach_debit_1m/(count_ach_credit_1m + 1) as ratio_ach_debit_credit_freq_1m, count_ach_debit_2m/(count_ach_credit_2m + 1) as ratio_ach_debit_credit_freq_2m,
amount_ach_debit_1m/(count_ach_debit_1m + 1) as ach_d_size_1m, amount_ach_debit_2m/(count_ach_debit_2m + 1) as ach_d_size_2m,
amount_ach_credit_1m/(count_ach_credit_1m + 1) as ach_c_size_1m, amount_ach_credit_2m/(count_ach_credit_2m + 1) as ach_c_size_2m

FROM (select distinct business_id, due_date, min(dateadd(month,-1,due_date)) over(partition by business_id) as min_created_at from PROD_DB.ADHOC.lending_dpd_data_till_may20) pay
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz on pay.business_id = biz.business_id
-- left join (select LENDING_BUSINESS_ID, min(created_at) as min_created_at, min(due_date) as first_due_date from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES"
-- group by 1) p on biz.id = p.lending_business_id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t on pay.business_id = t.business_id
where t.status = 'active'  AND  daysFromDueDate <=180 AND daysFromDueDate > 0 and DaysSinceLoanTaken >= 0
-- and pay.lending_business_id = '48cd97e3-f301-4958-bd0d-703ec133fb0b' 
order by transaction_date, pay.due_date
)
group by 1,2


""")

In [9]:
txn_df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m
0,d58f5d4b-6773-47ce-8424-7ebcdd180a6e,2023-03-03,0.0,23.0,9.0,80.0,1.0,0.0,41.0,18.0,133.0,2.0,0.0,4281.25,3021.13,4881.37,2993.63,0.00,6970.01,5343.00,8874.46,6883.61,0.0,178.385417,302.113000,60.263827,1496.815000,0.000000,165.952619,281.210526,66.227313,2294.536667,7188.01,13.0,513.429286,0.0,131.034537,16868.37,29.0,562.279000,0.0,214.479935,9216.62,105.0,86.949245,0.0,56.451128,17498.79,180.0,96.678398,0.0,82.123619,1.282043,7.500000,1.037311,6.000000,0.0,0.464514,0.420301,0.529627,0.416475,0.000000,0.398314,0.316747,0.507147,0.408078,0.0,0.216981,0.642857,0.754717,0.071429,0.000000,0.226519,0.600000,0.734807,0.066667,1.416633,1.304268,2.300000,2.157895,178.385417,165.952619,302.113000,281.210526
1,8dda3d56-0a47-4ab8-a14a-9d3c86dd21b4,2023-01-31,0.0,2.0,4.0,115.0,0.0,0.0,5.0,9.0,256.0,0.0,0.0,86.00,2995.00,3298.06,0.00,0.00,346.36,7975.00,7488.01,0.00,0.0,28.666667,599.000000,28.431552,0.000000,0.000000,57.726667,797.500000,29.136226,0.000000,5995.00,11.0,499.583333,0.0,72.229777,12545.47,25.0,482.518077,0.0,119.997856,5053.06,130.0,38.572977,0.0,26.192899,12635.53,297.0,42.401107,0.0,55.262402,0.842738,10.833333,1.007098,11.423077,0.0,0.017019,0.499583,0.652686,0.000000,0.000000,0.027412,0.635688,0.592615,0.000000,0.0,0.015267,0.333333,0.877863,0.000000,0.000000,0.016779,0.346154,0.859060,0.000000,0.028705,0.043425,0.400000,0.500000,28.666667,57.726667,599.000000,797.500000
2,e94b1103-281f-4b64-8791-7ad7978fbcdd,2023-03-08,0.0,16.0,3.0,58.0,1.0,1.0,42.0,6.0,168.0,2.0,0.0,1085.62,2942.84,3536.61,1250.00,10.71,3148.39,5895.50,11097.58,2500.00,0.0,63.860000,735.710000,59.942542,625.000000,5.355000,73.218372,842.214286,65.666154,833.333333,4205.34,5.0,700.890000,0.0,81.174670,13421.71,13.0,958.693571,0.0,152.785417,6736.73,83.0,80.199167,0.0,48.351657,19908.79,223.0,88.878527,0.0,110.297018,1.601566,13.833333,1.483217,15.928571,0.0,0.161149,0.699786,0.524974,0.297241,0.000798,0.158141,0.439251,0.557421,0.1

In [10]:
txn_df.shape

(7981, 84)

In [11]:
rb_df = querySno(r"""
with txns_post_lending as (
select  pay.business_id, pay.due_date, t.transaction_date,pay.min_created_at, amount, type, t.status, running_balance, 
datediff(day, pay.min_created_at, TRANSACTION_DATE) AS daySinceLoanTaken,
datediff(day, TRANSACTION_DATE, pay.due_date) AS daysFromDueDate
FROM (select distinct business_id, due_date, min(dateadd(month,-1,due_date)) over(partition by business_id) as min_created_at from PROD_DB.ADHOC.lending_dpd_data_till_may20) pay
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" biz on pay.business_id = biz.business_id
-- left join (select lending_business_id, min(created_at) as min_created_at from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_PAYMENT_SCHEDULES" group by 1) p 
-- on biz.id = p.id
left join FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" t on t.business_id = pay.business_id
where t.status = 'active'  AND  daySinceLoanTaken <=180 AND daySinceLoanTaken > 0 and daysFromDueDate > 0
-- and pay.lending_business_id = 'cf6d8b20-574f-4362-b1f5-274bb2231c05'
order by due_date
)
,cal_dates_1 AS (
			SELECT DATEADD(DAY, SEQ4(), '2021-06-01') AS CAL_DATE
			 FROM TABLE(GENERATOR(ROWCOUNT=>960)) 
			 ORDER BY CAL_DATE DESC),
biz_dates_1 AS (
			SELECT a.BUSINESS_ID, MIN(a.transaction_date) AS FIRST_TXN_DATE
			from FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS"  a 
			GROUP BY a.BUSINESS_ID),
PILOT_BIZ_DATES_TEMP AS (
		SELECT *
		FROM  cal_dates_1 A LEFT JOIN
		biz_dates_1 B ON
		date(A.CAL_DATE) >= date(B.FIRST_TXN_DATE)
		ORDER BY CAL_DATE ASC)
,PILOT_BIZ_TXN_TEMP AS (
		SELECT   Business_id ,TRANSACTION_DATE,running_balance
		FROM
		        (SELECT *, RANK () OVER (PARTITION BY BUSINESS_ID,TRANSACTION_DATE ORDER BY TRANSACTION_DATE DESC) RANKS
		         FROM FIVETRAN_DB.PROD_NOVO_API_PUBLIC."TRANSACTIONS" 
                 where 1=1
                 and status='active'
		         )
		WHERE RANKS=1
		order by Business_id, TRANSACTION_DATE)
,PILOT_DAILY_BALANCES_1  AS (
		SELECT BUSINESS_ID,FIRST_TXN_DATE, CAL_DATE,TRANSACTION_DATE,RUNNING_BALANCE,
		first_value(RUNNING_BALANCE) OVER (PARTITION BY BUSINESS_ID,GROUPER ORDER BY cal_date asc, TRANSACTION_DATE desc, RUNNING_BALANCE asc nulls last) as RUNNING_BALANCE_2
		FROM (
		SELECT A.BUSINESS_ID,A.CAL_DATE,A.FIRST_TXN_DATE,
		        B.TRANSACTION_DATE,B.RUNNING_BALANCE,
		         COUNT(running_balance) OVER (PARTITION BY a.BUSINESS_ID ORDER BY cal_date asc) as grouper
		        FROM PILOT_BIZ_DATES_TEMP A LEFT JOIN
		        PILOT_BIZ_TXN_TEMP B ON
		        A.BUSINESS_ID=B.BUSINESS_ID AND
		        date(A.CAL_DATE)= date(B.TRANSACTION_DATE)
		        ORDER BY A.CAL_DATE ASC)
		ORDER BY BUSINESS_ID,CAL_DATE)
,final_daily_running_balance AS (
		 SELECT distinct a.business_id, b.due_date, datediff(day, a.cal_date, b.due_date) as daysFromDueDate, cal_date, running_balance_2 AS running_balance_daily
		           FROM  PILOT_DAILY_BALANCES_1  a join txns_post_lending b 
		           on a.business_id = b.business_id
		          WHERE daysFromDueDate > 0 AND daysFromDueDate <= 180
		ORDER BY a.business_id, daysFromDueDate)
,oneMonth_rb AS (	
				SELECT business_id, due_date, min(cal_date) as min_txn_date_1mo_rb, max(cal_date) as max_txn_date_1mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_1m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_1m,
				               AVG(running_balance_daily) as Avg_running_balance_1m,
  							   Median(running_balance_daily) as Median_running_balance_1m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_1m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=30
				GROUP BY 1,2)
,twoMonth_rb AS (	
				SELECT business_id, due_date, min(cal_date) as min_txn_date_2mo_rb, max(cal_date) as max_txn_date_2mo_rb,
				               sum(CASE WHEN running_balance_daily < 0 THEN  1 ELSE 0 end) AS od_count_2m, 
				               sum(CASE WHEN running_balance_daily = 0 THEN  1 ELSE 0 end) AS zero_balance_count_2m,
				               AVG(running_balance_daily) as Avg_running_balance_2m,
  							   Median(running_balance_daily) as Median_running_balance_2m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_2m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=60
				GROUP BY 1,2)
				
,second_last_month_rb as (	
				SELECT business_id, due_date, 
				               AVG(running_balance_daily) as Avg_running_balance_2nd_m,
  							   Median(running_balance_daily) as Median_running_balance_2nd_m,
							   STDDEV(running_balance_daily) as STDDEV_running_balance_2nd_m
				FROM final_daily_running_balance
				WHERE daysFromDueDate <=60 and daysFromDueDate > 30
				GROUP BY 1,2)
-- select count(*) from (
select a.business_id, a.due_date, od_count_1m, zero_balance_count_1m, Avg_running_balance_1m, Median_running_balance_1m, STDDEV_running_balance_1m,
od_count_2m, zero_balance_count_2m, Avg_running_balance_2m, Median_running_balance_2m, STDDEV_running_balance_2m, Avg_running_balance_2nd_m, Median_running_balance_2nd_m, STDDEV_running_balance_2nd_m,
Avg_running_balance_1m/(Avg_running_balance_2nd_m + 1) as ratio_avg_running_balance_1M_2M
from oneMonth_rb a left join twoMonth_rb b on a.business_id = b.business_id and a.due_date = b.due_date
left join second_last_month_rb c on a.business_id = c.business_id and a.due_date = c.due_date
-- )
"""
)

In [12]:
rb_df.head()

,business_id,due_date,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m
0,1c8dc5fa-c1b6-44af-96f2-f13e553038a2,2023-05-13,0,0,50.570466,5.44,178.237070,0,3,87.395502,5.44,253.204112,429.868333,280.67,491.934567,0.117369
1,2253a00b-c479-460d-b4b9-26110517a2e7,2023-03-27,0,0,813.423865,860.75,213.258815,0,0,791.693652,860.75,236.159745,555.558667,456.03,331.732458,1.461524
2,a9a77d6b-cbd2-4ee3-9d97-3ba57aea0f4c,2023-03-18,0,0,4926.442418,3510.32,2556.513676,0,0,5111.746767,3510.32,2532.868737,7180.978667,7219.67,651.656621,0.685945
3,6fb7feaf-c792-4e22-b3c7-6a41e3f39bb4,2023-01-07,10,302,21.120025,0.00,55.550750,12,302,101.618713,0.00,458.790104,1188.351000,329.39,1339.727487,0.017758
4,a3a69fd9-4697-4226-be33-1c90d89f89ef,2023-02-25,0,0,1737.801433,1613.22,477.038498,0,0,1869.865829,1613.22,678.355124,3437.030000,3548.59,754.043062,0.505464


In [13]:
rb_df.shape

(7971, 16)

In [14]:
mom_df = querySno(r"""
select * from 
(
select business_id, due_date, credit_limit, avg(amount_drawn_1m_prev)amount_drawn_1m_prev, avg(amount_drawn_2m_prev)amount_drawn_2m_prev, avg(amount_cashin_1m_prev)amount_cashin_1m_prev, 
avg(amount_cashin_2m_prev)amount_cashin_2m_prev, avg(distinct_draws_1m_prev)distinct_draws_1m_prev, avg(distinct_draws_2m_prev)distinct_draws_2m_prev, avg(distinct_cashins_1m_prev)distinct_cashins_1m_prev, avg(distinct_cashins_2m_prev)distinct_cashins_2m_prev,
avg(ratio_amt_drawn_mom)ratio_amt_drawn_mom, avg(ratio_amt_cashin_mom)ratio_amt_cashin_mom, avg(ratio_count_drawn_mom)ratio_count_drawn_mom, avg(ratio_count_cashin_mom)ratio_count_cashin_mom
from (
select b.lending_business_id,a.business_id, due_date,b.created_at,credit_limit/100 as credit_limit , datediff(month, b.created_at, due_date) as txn_month, b.amount, type,
sum(case when txn_month = 0 and type = 'draw' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_drawn_1m_prev,
sum(case when txn_month = 1 and type = 'draw' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_drawn_2m_prev,
sum(case when txn_month = 0 and type = 'cash_in' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_cashin_1m_prev,
sum(case when txn_month = 1 and type = 'cash_in' then amount/100 else 0 end) over(partition by b.lending_business_id, due_date) as amount_cashin_2m_prev,

count(distinct case when txn_month = 0 and type = 'draw' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_draws_1m_prev,
count(distinct case when txn_month = 1 and type = 'draw' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_draws_2m_prev,
count(distinct case when txn_month = 0 and type = 'cash_in' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_cashins_1m_prev,
count(distinct case when txn_month = 1 and type = 'cash_in' then trace_number end) over(partition by b.lending_business_id, due_date) as distinct_cashins_2m_prev,

amount_drawn_1m_prev/(amount_drawn_2m_prev + 1) as ratio_amt_drawn_mom, 
amount_cashin_1m_prev/(amount_cashin_2m_prev + 1) as ratio_amt_cashin_mom, 
distinct_draws_1m_prev/(distinct_draws_2m_prev + 1) as ratio_count_drawn_mom, 
distinct_cashins_1m_prev/(distinct_cashins_2m_prev + 1) as ratio_count_cashin_mom

from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_TRANSACTIONS" b 
join 
(select distinct business_id, due_date from PROD_DB.ADHOC.lending_dpd_data_till_may20) a
on a.business_id = b.business_id
left join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_BUSINESSES" c on b.lending_business_id = c.id
where txn_month >= 0 and txn_month <2
-- and b.lending_business_id = '005da9f7-5939-4d37-bac8-4956a2a9e419'
order by due_date
)
group by 1,2,3
order by 1,2,3
)

UNION

(
select business_id, due_date, credit_limit, avg(amount_drawn_1m_prev)amount_drawn_1m_prev, avg(amount_drawn_2m_prev)amount_drawn_2m_prev, avg(amount_cashin_1m_prev)amount_cashin_1m_prev, 
avg(amount_cashin_2m_prev)amount_cashin_2m_prev, avg(distinct_draws_1m_prev)distinct_draws_1m_prev, avg(distinct_draws_2m_prev)distinct_draws_2m_prev, avg(distinct_cashins_1m_prev)distinct_cashins_1m_prev, avg(distinct_cashins_2m_prev)distinct_cashins_2m_prev,
avg(ratio_amt_drawn_mom)ratio_amt_drawn_mom, avg(ratio_amt_cashin_mom)ratio_amt_cashin_mom, avg(ratio_count_drawn_mom)ratio_count_drawn_mom, avg(ratio_count_cashin_mom)ratio_count_cashin_mom
from (
select p.business_id, b.due_date,a.created_at,p.credit_limit , datediff(month, t.transaction_date, b.due_date) as txn_month, t.amount, a.action_type,
sum(case when txn_month = 0 and action_type = 'DRAW' then t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_drawn_1m_prev,
sum(case when txn_month = 1 and action_type = 'DRAW' then t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_drawn_2m_prev,
sum(case when txn_month = 0 and action_type = 'CASHIN' then -t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_cashin_1m_prev,
sum(case when txn_month = 1 and action_type = 'CASHIN' then -t.amount else 0 end) over(partition by b.business_id, b.due_date) as amount_cashin_2m_prev,

count(distinct case when txn_month = 0 and action_type = 'DRAW' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_draws_1m_prev,
count(distinct case when txn_month = 1 and action_type = 'DRAW' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_draws_2m_prev,
count(distinct case when txn_month = 0 and action_type = 'CASHIN' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_cashins_1m_prev,
count(distinct case when txn_month = 1 and action_type = 'CASHIN' then a.transaction_id end) over(partition by b.business_id, b.due_date) as distinct_cashins_2m_prev,

amount_drawn_1m_prev/(amount_drawn_2m_prev + 1) as ratio_amt_drawn_mom, 
amount_cashin_1m_prev/(amount_cashin_2m_prev + 1) as ratio_amt_cashin_mom, 
distinct_draws_1m_prev/(distinct_draws_2m_prev + 1) as ratio_count_drawn_mom, 
distinct_cashins_1m_prev/(distinct_cashins_2m_prev + 1) as ratio_count_cashin_mom

from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LOAN_PRODUCT_TRANSACTIONS a 
join "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LOAN_PRODUCTS" p on a.product_id = p.id
join 
(select distinct business_id, due_date from PROD_DB.ADHOC.lending_dpd_data_till_may20
where due_date < date('2023-01-01')) b 
on p.business_id = b.business_id
left join prod_db.data.transactions t on a.transaction_id = t.transaction_id
where txn_month >= 0 and txn_month <2
-- and b.lending_business_id = '005da9f7-5939-4d37-bac8-4956a2a9e419'
order by due_date
)
group by 1,2,3
order by 1,2,3
)
"""
)

In [15]:
mom_df.head()

,business_id,due_date,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,1fbe7d67-50d2-4117-bbf1-1722cf535615,2023-04-13,8400.0,1900.0,5900.0,1492.11,546.57,5.0,9.0,5.0,3.0,0.321979,2.724967,0.500000,1.250000
1,1feef942-56ae-473a-8e55-a9c6c64c2f05,2023-01-08,10000.0,0.0,10000.0,4235.24,0.00,0.0,2.0,4.0,0.0,0.000000,4235.240000,0.000000,4.000000
2,201e1496-9675-41f6-9200-233d3e4cb7d8,2023-01-02,10000.0,0.0,8600.0,1648.33,0.00,0.0,1.0,1.0,0.0,0.000000,1648.330000,0.000000,1.000000
3,20ac416b-493e-4113-acb2-c4ff3835a814,2023-05-15,37500.0,0.0,10100.0,5618.66,3733.33,0.0,1.0,1.0,1.0,0.000000,1.504597,0.000000,0.500000
4,211d7a9e-185b-4f9e-a8df-ddbad154e692,2023-04-23,6800.0,1800.0,900.0,1270.48,887.32,1.0,2.0,5.0,2.0,1.997780,1.430205,0.333333,1.666667


In [16]:
mom_df.shape

(8025, 15)

In [17]:
mom_df = mom_df.drop_duplicates(subset = ['business_id','due_date'])

In [18]:
print (rev_df.shape)
print (txn_df.shape)
print (rb_df.shape)
print (mom_df.shape)

(7913, 6)
(7981, 84)
(7971, 16)
(7868, 15)


In [19]:
df1 = txn_df.merge(rb_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])
df2 = df1.merge(rev_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])
df = df2.merge(mom_df,how='inner',left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])

In [20]:
df.shape

(7802, 115)

In [21]:
df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,d58f5d4b-6773-47ce-8424-7ebcdd180a6e,2023-03-03,0.0,23.0,9.0,80.0,1.0,0.0,41.0,18.0,133.0,2.0,0.0,4281.25,3021.13,4881.37,2993.63,0.00,6970.01,5343.00,8874.46,6883.61,0.0,178.385417,302.113000,60.263827,1496.815000,0.000000,165.952619,281.210526,66.227313,2294.536667,7188.01,13.0,513.429286,0.0,131.034537,16868.37,29.0,562.279000,0.0,214.479935,9216.62,105.0,86.949245,0.0,56.451128,17498.79,180.0,96.678398,0.0,82.123619,1.282043,7.500000,1.037311,6.000000,0.0,0.464514,0.420301,0.529627,0.416475,0.000000,0.398314,0.316747,0.507147,0.408078,0.0,0.216981,0.642857,0.754717,0.071429,0.000000,0.226519,0.600000,0.734807,0.066667,1.416633,1.304268,2.300000,2.157895,178.385417,165.952619,302.113000,281.210526,9,6,233.251029,8.03,856.087717,9,6,293.205921,8.03,919.079074,992.679667,227.510,1292.290350,0.234735,4393.48,7188.01,11581.49,0.611138,7000.0,1600.0,0.0,2230.95,0.00,1.0,0.0,10.0,0.0,1600.000000,2230.950000,1.0,10.000000
1,8dda3d56-0a47-4ab8-a14a-9d3c86dd21b4,2023-01-31,0.0,2.0,4.0,115.0,0.0,0.0,5.0,9.0,256.0,0.0,0.0,86.00,2995.00,3298.06,0.00,0.00,346.36,7975.00,7488.01,0.00,0.0,28.666667,599.000000,28.431552,0.000000,0.000000,57.726667,797.500000,29.136226,0.000000,5995.00,11.0,499.583333,0.0,72.229777,12545.47,25.0,482.518077,0.0,119.99

In [22]:
df.describe()

,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
count,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7.802000e+03,7802.000000,7802.000000,7802.000000,7802.000000,7.802000e+03,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7.802000e+03,7.802000e+03,7.802000e+03,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000000,7802.000

In [23]:
null_df = pd.DataFrame(df.isnull().sum().sort_values(ascending=False),columns = ['null_count'])
null_df = null_df[null_df['null_count']>0]

In [24]:
null_df.shape

(0, 1)

In [ ]:
# df = df.fillna(0)

In [25]:
base_df = querySno(r"""
select * from PROD_DB.ADHOC.lending_dpd_data_till_may20
""")

In [26]:
base_df.shape

(8003, 6)

In [27]:
base_df.sample(5)

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
1638,175509dc-2b5d-4caa-ade2-70b0a984ee35,2023-02-27,0,0,0,1
3114,d697115b-73b6-4720-9fa6-507e59c7c7ca,2023-04-15,0,0,0,2
2574,261b95a5-850e-4878-b220-2458d078e06b,2023-05-13,0,0,0,2
2212,3837b607-058b-4772-afc5-bcb26a48893e,2023-01-13,1,1,1,4
5354,6af33b38-13e3-4091-ac3a-78928f4a4c94,2023-04-12,1,1,0,3


In [28]:
base_df = base_df.drop_duplicates(subset = ['business_id','due_date'])

In [29]:
base_df['DPD_plus_15'] = base_df['DPD_plus_15'].astype(str).astype(int)
base_df['DPD_plus_3'] = base_df['DPD_plus_3'].astype(str).astype(int)
base_df['emi_no'] = base_df['emi_no'].astype(str).astype(int)

In [30]:
base_df.head()

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no
0,9c8d5541-61ef-4203-ad12-d0a79722f4f2,2023-05-19,0,0,0,1
1,20f98713-f8e1-4a53-849e-4b2fcab7b0ab,2023-05-19,0,0,0,1
2,070f371f-40a5-4432-999c-38bfe18339dd,2023-05-19,0,0,0,1
3,97656f70-5038-4f29-af8a-eb15450e9736,2023-05-19,0,0,0,1
4,d6085e7a-e8f6-4a9a-9bed-bef96b31f43b,2023-05-19,0,0,0,1


In [31]:
b = base_df.copy()

In [32]:
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(-1).rename('DPD_plus_3_next'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(1).rename('DPD_plus_3_prev'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_3'].shift(2).rename('DPD_plus_3_prev_prev'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(-1).rename('DPD_plus_15_next'),left_index=True,right_index=True)
base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(1).rename('DPD_plus_15_prev'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby(['business_id'])['DPD_plus_15'].shift(2).rename('DPD_plus_15_prev_prev'),left_index=True,right_index=True)

In [33]:
base_df = base_df.fillna(0)

In [34]:
base_df['DPD_plus_3_last_2'] = base_df['DPD_plus_3_prev'] + base_df['DPD_plus_3']
base_df['DPD_plus_15_last_2'] = base_df['DPD_plus_15_prev'] + base_df['DPD_plus_15']

In [ ]:
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby('business_id')['DPD_plus_3_prev'].cumsum().rename('DPD_plus_3_prev_cumsum'),left_index=True,right_index=True)
# base_df = base_df.merge(base_df.sort_values(['business_id','emi_no']).groupby('business_id')['DPD_plus_15_prev'].cumsum().rename('DPD_plus_15_prev_cumsum'),left_index=True,right_index=True)

In [ ]:
# base_df['DPD_plus_3_prev_ratio'] = base_df['DPD_plus_3_prev_cumsum']/(base_df['emi_no']-1)
# base_df['DPD_plus_15_prev_ratio'] = base_df['DPD_plus_15_prev_cumsum']/(base_df['emi_no']-1)
# base_df = base_df.fillna(0)

In [35]:
base_df[base_df['business_id']=='1724f6de-5ba9-45a7-917d-8a0e575b6297'].sort_values('emi_no')

,business_id,due_date,DPD_plus_3,DPD_plus_15,DPD_plus_30,emi_no,DPD_plus_3_next,DPD_plus_3_prev,DPD_plus_15_next,DPD_plus_15_prev,DPD_plus_3_last_2,DPD_plus_15_last_2
6283,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-09-10,0,0,0,1,0.0,0.0,0.0,0.0,0.0,0.0
6926,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-09-20,0,0,0,2,0.0,0.0,0.0,0.0,0.0,0.0
7569,1724f6de-5ba9-45a7-917d-8a0e575b6297,2022-10-10,0,0,0,3,0.0,0.0,0.0,0.0,0.0,0.0
2175,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-01-21,0,0,0,4,1.0,0.0,0.0,0.0,0.0,0.0
4102,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-02-21,1,0,0,5,0.0,0.0,0.0,0.0,1.0,0.0
5492,1724f6de-5ba9-45a7-917d-8a0e575b6297,2023-03-21,0,0,0,6,0.0,1.0,0.0,0.0,1.0,0.0


In [36]:
bdf = base_df[['business_id','due_date','DPD_plus_15_next','DPD_plus_3_last_2','DPD_plus_15_prev','emi_no','DPD_plus_3']]

In [37]:
df.head()

,business_id,due_date,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,d58f5d4b-6773-47ce-8424-7ebcdd180a6e,2023-03-03,0.0,23.0,9.0,80.0,1.0,0.0,41.0,18.0,133.0,2.0,0.0,4281.25,3021.13,4881.37,2993.63,0.00,6970.01,5343.00,8874.46,6883.61,0.0,178.385417,302.113000,60.263827,1496.815000,0.000000,165.952619,281.210526,66.227313,2294.536667,7188.01,13.0,513.429286,0.0,131.034537,16868.37,29.0,562.279000,0.0,214.479935,9216.62,105.0,86.949245,0.0,56.451128,17498.79,180.0,96.678398,0.0,82.123619,1.282043,7.500000,1.037311,6.000000,0.0,0.464514,0.420301,0.529627,0.416475,0.000000,0.398314,0.316747,0.507147,0.408078,0.0,0.216981,0.642857,0.754717,0.071429,0.000000,0.226519,0.600000,0.734807,0.066667,1.416633,1.304268,2.300000,2.157895,178.385417,165.952619,302.113000,281.210526,9,6,233.251029,8.03,856.087717,9,6,293.205921,8.03,919.079074,992.679667,227.510,1292.290350,0.234735,4393.48,7188.01,11581.49,0.611138,7000.0,1600.0,0.0,2230.95,0.00,1.0,0.0,10.0,0.0,1600.000000,2230.950000,1.0,10.000000
1,8dda3d56-0a47-4ab8-a14a-9d3c86dd21b4,2023-01-31,0.0,2.0,4.0,115.0,0.0,0.0,5.0,9.0,256.0,0.0,0.0,86.00,2995.00,3298.06,0.00,0.00,346.36,7975.00,7488.01,0.00,0.0,28.666667,599.000000,28.431552,0.000000,0.000000,57.726667,797.500000,29.136226,0.000000,5995.00,11.0,499.583333,0.0,72.229777,12545.47,25.0,482.518077,0.0,119.99

In [38]:
dff = bdf.merge(df,how='inner', left_on = ['business_id','due_date'], right_on = ['business_id','due_date'])

In [39]:
dff.shape

(7802, 120)

In [40]:
dff['emi_no'].value_counts()

1    2750
2    2173
3    1623
4     616
5     331
6     309
Name: emi_no, dtype: int64

In [41]:
dff.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,9c8d5541-61ef-4203-ad12-d0a79722f4f2,2023-05-19,0.0,0.0,0.0,1,0,0.0,1.0,2.0,42.0,0.0,0.0,1.0,2.0,42.0,0.0,0.00,49.16,16309.71,1967.84,0.0,0.00,49.16,16309.71,1967.84,0.0,0.000000,24.580000,5436.570000,45.763721,0.000000,0.000000,24.580000,5436.570000,45.763721,0.000000,37109.71,3.0,9277.427500,0.0,3669.903201,37109.71,3.0,9277.427500,0.0,3669.903201,39324.50,47.0,819.260417,27.805,3665.342990,39324.50,47.0,819.260417,27.805,3665.342990,1.059654,11.750000,1.059654,11.750000,0.000000,0.001250,0.439500,0.050041,0.00,0.000000,0.001250,0.439500,0.050041,0.00,0.000000,0.020833,0.500000,0.875000,0.0,0.000000,0.020833,0.500000,0.875000,0.0,0.003014,0.003014,0.333333,0.333333,24.580000,24.580000,5436.570000,5436.570000,0,241,113.409158,0.00,386.711396,0,241,613.054917,0.00,1800.925296,5159.831333,4061.90,2938.808165,0.021975,349.71,15960.00,16309.71,0.021910,20800.0,0.0,20800.0,71.25,0.00,0.0,1.0,2.0,0.0,0.000000,71.25,0.0,2.0
1,20f98713-f8e1-4a53-849e-4b2fcab7b0ab,2023-05-19,0.0,0.0,0.0,1,0,0.0,1.0,0.0,8.0,2.0,0.0,1.0,0.0,8.0,2.0,0.00,209.00,0.00,484.90,2000.0,0.00,209.00,0.00,484.90,2000.0,0.000000,104.500000,0.000000,53.877778,666.666667,0.000000,104.500000,0.000000,53.8777

In [42]:
dff.groupby('emi_no')['DPD_plus_15_next'].sum()

emi_no
1    138.0
2    129.0
3     36.0
4     60.0
5     34.0
6      0.0
Name: DPD_plus_15_next, dtype: float64

In [44]:
import plotly.express as px
fig = px.bar(dff.groupby(['DPD_plus_3'])['DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400)
fig.update_traces(width=0.4)
fig.show()

In [45]:
dff_copy = dff.copy()

In [51]:
dff.shape

(7802, 120)

In [46]:
dff.head()

,business_id,due_date,DPD_plus_15_next,DPD_plus_3_last_2,DPD_plus_15_prev,emi_no,DPD_plus_3,count_pd_deposit_1m,count_ach_debit_1m,count_ach_credit_1m,count_card_withdrawal_1m,count_mrdc_credit_1m,count_pd_deposit_2m,count_ach_debit_2m,count_ach_credit_2m,count_card_withdrawal_2m,count_mrdc_credit_2m,amount_pd_deposit_1m,amount_ach_debit_1m,amount_ach_credit_1m,amount_card_withdrawal_1m,amount_mrdc_credit_1m,amount_pd_deposit_2m,amount_ach_debit_2m,amount_ach_credit_2m,amount_card_withdrawal_2m,amount_mrdc_credit_2m,pd_deposit_size_1m,ach_debit_size_1m,ach_credit_size_1m,card_withdrawal_size_1m,mrdc_credit_size_1m,pd_deposit_size_2m,ach_debit_size_2m,ach_credit_size_2m,card_withdrawal_size_2m,mrdc_credit_size_2m,amount_credited_1m,distinct_credit_txns_1m,credit_size_1m,median_amount_credited_1m,stddev_amount_credited_1m,amount_credited_2m,distinct_credit_txns_2m,credit_size_2m,median_amount_credited_2m,stddev_amount_credited_2m,amount_debited_1m,distinct_debit_txns_1m,debit_size_1m,median_amount_debited_1m,stddev_amount_debited_1m,amount_debited_2m,distinct_debit_txns_2m,debit_size_2m,median_amount_debited_2m,stddev_amount_debited_2m,ratio_debit_credit_1m,ratio_debit_credit_frequency_1m,ratio_debit_credit_2m,ratio_debit_credit_frequency_2m,ratio_pd_amt_total_credit_1m,ratio_ach_debit_total_debit_1m,ratio_ach_credit_total_credit_1m,ratio_card_withdrawal_total_debit_1m,ratio_mrdc_credit_total_credit_1m,ratio_pd_amt_total_credit_2m,ratio_ach_debit_total_debit_2m,ratio_ach_credit_total_credit_2m,ratio_card_withdrawal_total_debit_2m,ratio_mrdc_credit_total_credit_2m,ratio_pd_freq_total_credit_1m,ratio_ach_debit_freq_total_debit_1m,ratio_ach_credit_freq_total_credit_1m,ratio_card_withdrawal_freq_total_debit_1m,ratio_mrdc_credit_freq_total_credit_1m,ratio_pd_freq_total_credit_2m,ratio_ach_debit_freq_total_debit_2m,ratio_ach_credit_freq_total_credit_2m,ratio_card_withdrawal_freq_total_debit_2m,ratio_mrdc_credit_freq_total_credit_2m,ratio_ach_debit_credit_1m,ratio_ach_debit_credit_2m,ratio_ach_debit_credit_freq_1m,ratio_ach_debit_credit_freq_2m,ach_d_size_1m,ach_d_size_2m,ach_c_size_1m,ach_c_size_2m,od_count_1m,zero_balance_count_1m,avg_running_balance_1m,median_running_balance_1m,stddev_running_balance_1m,od_count_2m,zero_balance_count_2m,avg_running_balance_2m,median_running_balance_2m,stddev_running_balance_2m,avg_running_balance_2nd_m,median_running_balance_2nd_m,stddev_running_balance_2nd_m,ratio_avg_running_balance_1m_2m,revenue_past_1m,revenue_past_2m,total_revenue_2m,ratio_revenue_1m_2m,credit_limit,amount_drawn_1m_prev,amount_drawn_2m_prev,amount_cashin_1m_prev,amount_cashin_2m_prev,distinct_draws_1m_prev,distinct_draws_2m_prev,distinct_cashins_1m_prev,distinct_cashins_2m_prev,ratio_amt_drawn_mom,ratio_amt_cashin_mom,ratio_count_drawn_mom,ratio_count_cashin_mom
0,9c8d5541-61ef-4203-ad12-d0a79722f4f2,2023-05-19,0.0,0.0,0.0,1,0,0.0,1.0,2.0,42.0,0.0,0.0,1.0,2.0,42.0,0.0,0.00,49.16,16309.71,1967.84,0.0,0.00,49.16,16309.71,1967.84,0.0,0.000000,24.580000,5436.570000,45.763721,0.000000,0.000000,24.580000,5436.570000,45.763721,0.000000,37109.71,3.0,9277.427500,0.0,3669.903201,37109.71,3.0,9277.427500,0.0,3669.903201,39324.50,47.0,819.260417,27.805,3665.342990,39324.50,47.0,819.260417,27.805,3665.342990,1.059654,11.750000,1.059654,11.750000,0.000000,0.001250,0.439500,0.050041,0.00,0.000000,0.001250,0.439500,0.050041,0.00,0.000000,0.020833,0.500000,0.875000,0.0,0.000000,0.020833,0.500000,0.875000,0.0,0.003014,0.003014,0.333333,0.333333,24.580000,24.580000,5436.570000,5436.570000,0,241,113.409158,0.00,386.711396,0,241,613.054917,0.00,1800.925296,5159.831333,4061.90,2938.808165,0.021975,349.71,15960.00,16309.71,0.021910,20800.0,0.0,20800.0,71.25,0.00,0.0,1.0,2.0,0.0,0.000000,71.25,0.0,2.0
1,20f98713-f8e1-4a53-849e-4b2fcab7b0ab,2023-05-19,0.0,0.0,0.0,1,0,0.0,1.0,0.0,8.0,2.0,0.0,1.0,0.0,8.0,2.0,0.00,209.00,0.00,484.90,2000.0,0.00,209.00,0.00,484.90,2000.0,0.000000,104.500000,0.000000,53.877778,666.666667,0.000000,104.500000,0.000000,53.8777

#### Adding RS1 & RS2 bins

In [47]:
ddf = querySno(r"""
select json_extract_path_text(predict_meta, 'business_id') as business_id, json_extract_path_text(predict_meta, 'default_bin') as rs2_bin , novo_risk_score as rs2_score , fico_score
from FIVETRAN_DB.PROD_NOVO_API_PUBLIC.LENDING_DECISION_RESULTS a 
left join  "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."CUSTOMER_CREDIT_PROFILES" b 
on a.lending_business_id = b.lending_business_id
""")



In [48]:
ldf = querySno(r""" 
select business_id, rs1_bin as rs1_bin_l, rs1_score as rs1_score_l, rs2_bin as rs2_bin_l, rs2_score as rs2_score_l from PROD_DB.ADHOC.LENDING_PRESCREEN_2_DEC_26TH
"""
)

In [ ]:
# ldf = querySno(r""" 
# select business_id, rs2_bin, rs2_score from PROD_DB.ADHOC.LENDING_PRESCREEN_2_DEC_26TH
# where business_id in {}
# """.format(tuple(dff.loc[dff['rs2_bin'].isnull(),'business_id']))
# )

In [49]:
ddf2 = querySno(r""" 
select *, novo_risk_score rs1_score
from (
select *, row_number() over(partition by business_id order by offer_start_at desc) rnk from "FIVETRAN_DB"."PROD_NOVO_API_PUBLIC"."LENDING_INVITATIONS"
)
where rnk = 1

""")

In [50]:
print(ddf.shape)
print(ldf.shape)
print(ddf2.shape)

(9766, 4)
(186075, 5)
(46155, 24)


In [ ]:
ddf = ddf.drop_duplicates(subset=['business_id'])
ldf = ldf.drop_duplicates(subset=['business_id'])
ddf2 = ddf2.drop_duplicates(subset=['business_id'])

In [ ]:
ddf = ddf[~ddf['business_id'].isnull()]

In [ ]:
ddf.head()

In [ ]:
ddf2 = ddf2[['business_id','rs1_score']]

In [ ]:
def rs1_bin(x):
    try: x = float(x)
    except: x = np.nan
    if x >= 656:
        return 5
    if x >= 545:
        return 4
    if x >= 489:
        return 3
    if x >= 443:
        return 2
    return 1

In [ ]:
ddf2['rs1_bin'] = ddf2['rs1_score'].apply(lambda x: rs1_bin(x))

In [ ]:
ddf2.head()

In [ ]:
df0 = ddf.merge(ddf2,left_on='business_id', right_on='business_id')

In [ ]:
df0.shape

In [ ]:
df0.head()

In [ ]:
dff.head()

In [ ]:
dff = dff.merge(df0, how='left', left_on='business_id', right_on = 'business_id')

In [ ]:
temp = dff.merge(ldf, how='left', left_on='business_id', right_on = 'business_id')

In [ ]:
temp.head()

In [ ]:
temp['rs2_bin'] = temp['rs2_bin'].fillna(temp['rs2_bin_l'])
temp['rs2_score'] = temp['rs2_score'].fillna(temp['rs2_score_l'])
temp['rs1_bin'] = temp['rs1_bin'].fillna(temp['rs1_bin_l'])
temp['rs1_score'] = temp['rs1_score'].fillna(temp['rs1_score_l'])

In [ ]:
temp[['rs2_bin','rs1_bin']] = temp[['rs2_bin','rs1_bin']].astype(int)
temp[['rs2_score','rs1_score']] = temp[['rs2_score','rs1_score']].astype(float)

In [ ]:
dff['fico_score'] = dff['fico_score'].fillna(0).replace('',0).astype(float)

In [ ]:
temp[['rs1_bin','rs1_score','rs2_bin','rs2_score']].isnull().sum()

In [ ]:
dff = temp[list(dff.columns)]

In [ ]:
dff.head()

In [ ]:
dff['fico_score'] = dff['fico_score'].fillna(0).replace('',0).astype(float)

In [ ]:
dff['fico_score'].describe()

In [ ]:
dff.head()

In [ ]:
import plotly.express as px
fig = px.bar(dff.groupby(['rs2_bin'])['DPD_plus_15_next'].sum(),text_auto=True,width=800, height=400)
fig.update_traces(width=0.4)
fig.show()

In [ ]:
pd.crosstab(dff['rs2_bin'],dff['emi_no'])

In [ ]:
pd.pivot_table(dff[['rs2_bin','emi_no','DPD_plus_15_next']],index = 'rs2_bin',
               columns = 'emi_no',aggfunc ='sum').to_clipboard()

In [ ]:
# tuple(dff.loc[dff['rs2_bin'].isnull(),'business_id'])

In [ ]:
fig = px.imshow(pd.pivot_table(dff[['rs2_bin','emi_no','DPD_plus_15_next']],index = 'rs2_bin',
               columns = 'emi_no',aggfunc ='sum'), text_auto=True, width=800, height=400)
fig.show()

In [ ]:
dff.groupby(['rs2_bin','emi_no'])['DPD_plus_15_next'].sum()

#### Saving data

In [54]:
dff.columns

Index(['business_id', 'due_date', 'DPD_plus_15_next', 'DPD_plus_3_last_2',
       'DPD_plus_15_prev', 'emi_no', 'DPD_plus_3', 'count_pd_deposit_1m',
       'count_ach_debit_1m', 'count_ach_credit_1m',
       ...
       'amount_cashin_1m_prev', 'amount_cashin_2m_prev',
       'distinct_draws_1m_prev', 'distinct_draws_2m_prev',
       'distinct_cashins_1m_prev', 'distinct_cashins_2m_prev',
       'ratio_amt_drawn_mom', 'ratio_amt_cashin_mom', 'ratio_count_drawn_mom',
       'ratio_count_cashin_mom'],
      dtype='object', length=120)

In [55]:
from score_funcs import Score
sc = Score()

/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning
/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Code


In [56]:
cols = ['business_id', 'emi_no','DPD_plus_15_next'] + sc.features

In [57]:
dff_out = dff[cols]

In [58]:
dff_out.shape

(7802, 13)

In [60]:
# dff_out.to_pickle("/Users/shashankgupta/Documents/code/git_project/redec/re_decisioning/Data/dataset_till_may20.pkl")